In [4]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr

In [170]:
Predicted = pd.read_csv("./Stereoscope/Outputs/output_Sim8-6.csv")
Target=pd.read_csv("./Data/Sim8/target.csv")

In [171]:
Predicted

,Unnamed: 0,Connecting Tubule,Distal Convoluted Tubule,Endothelial Cell,Intercalated Cells Type A,Intercalated Cells Type B,Leukocyte,Mesangial Cells,Principal Cells,Proximal Tubule,Thick Ascending Limb
0,0,0.141749,0.024078,0.021597,0.314721,0.024728,1.696188e-02,2.103009e-05,0.226281,0.229860,0.000001
1,1,0.000015,0.000003,0.045912,0.125576,0.086958,5.192670e-02,4.176595e-07,0.119359,0.123057,0.447193
2,2,0.006474,0.116550,0.000003,0.000002,0.000004,5.819594e-02,1.815969e-01,0.120468,0.516703,0.000003
3,3,0.398113,0.025431,0.132558,0.038922,0.000017,8.734045e-07,3.868237e-02,0.000005,0.366268,0.000002
4,4,0.069390,0.000002,0.202154,0.127790,0.000002,2.384352e-02,1.151505e-06,0.475305,0.096496,0.005016
...,...,...,...,...,...,...,...,...,...,...,...
995,995,0.000367,0.000002,0.184585,0.107890,0.010894,1.852022e-06,4.939870e-02,0.061126,0.585734,0.000002
996,996,0.701285,0.048276,0.000002,0.000002,0.000001,3.813710e-06,1.144979e-06,0.000011,0.131454,0.118963
997,997,0.383534,0.000004,0.084149,0.071334,0.078473,9.918743e-03,4.578670e-02,0.224591,0.086416,0.015793
998,998,0.000009,0.069548,0.374764,0.000002,0.113245,4.331160e-02,2.052043e-02,0.122701,0.255893,0.000007


In [172]:
Target

,Unnamed: 0,Proximal Tubule,Thick Ascending Limb,Distal Convoluted Tubule,Connecting Tubule,Principal Cells,Intercalated Cells Type A,Intercalated Cells Type B,Endothelial Cell,Mesangial Cells,Leukocyte
0,0,5.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0
1,1,5.0,4.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,2,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
995,995,9.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
996,996,2.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
997,997,3.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0
998,998,9.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [173]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
#Predicted.drop(columns="cell_ID", inplace=True)


In [174]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [175]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [176]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [177]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [178]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [179]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [180]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [181]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [182]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [183]:
np.mean(np.nan_to_num(d))

0.7090322331118711

In [184]:
d

array([0.53997045, 0.82699691, 0.8377121 , 0.78434654, 0.55225154,
       0.31049148, 0.71577115, 0.82258177, 0.82377345, 0.87642694])

## Jenson Shannon Divergence

In [185]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.3642915324979307

## F1 Score, Precision, Recall

In [186]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [187]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [188]:
d

,Precision,Recall,F1-score
0.100,0.718663,0.830175,0.770405
0.200,0.808319,0.648040,0.719360
0.300,0.773774,0.501297,0.608422
0.010,0.421517,0.966381,0.586997
0.001,0.401845,0.971184,0.568474


## KL Divergence